In [1]:
!pip install docling[pdf]

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ------------------ --------------------- 7.3/16.1 MB 90.7 MB/s eta 0:00:01
   ---------------------------------------  16.0/16.1 MB 41.9 MB/s eta 0:00:01
   ---------------------------------------  16.0/16.1 MB 41.9 MB/s eta 0:00:01
   ---------------------------------------- 16.1/16.1 MB 23.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 84.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 28.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 10.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   --

In [ ]:
# import requests
# import tempfile
# import os
# import fitz  # PyMuPDF

# def extract_text_safe(pdf_url):
#     """
#     Safe PDF text extraction with better error handling
#     """
#     temp_path = None
#     doc = None
    
#     try:
#         print("📥 Downloading PDF...")
#         response = requests.get(pdf_url, timeout=60)
#         response.raise_for_status()
        
#         # Save to temporary file
#         with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as f:
#             f.write(response.content)
#             temp_path = f.name
        
#         print("🔍 Opening PDF document...")
#         # Open the document
#         doc = fitz.open(temp_path)
#         total_pages = len(doc)
#         print(f"📄 Found {total_pages} pages")
        
#         # Process in smaller chunks to avoid memory issues
#         text = ""
#         pages_to_process = min(5, total_pages)  # Start with just 5 pages
        
#         print(f"🔄 Processing first {pages_to_process} pages...")
#         for page_num in range(pages_to_process):
#             print(f"   Processing page {page_num + 1}...")
#             page = doc[page_num]
#             page_text = page.get_text()
#             text += f"\n--- Page {page_num + 1} ---\n"
#             text += page_text
        
#         # Close document explicitly
#         doc.close()
#         doc = None
        
#         # Clean up temporary file
#         if temp_path and os.path.exists(temp_path):
#             os.unlink(temp_path)
#             temp_path = None
        
#         return {
#             'success': True,
#             'text': text,
#             'pages_processed': pages_to_process,
#             'total_pages': total_pages
#         }
        
#     except Exception as e:
#         print(f"❌ Error occurred: {str(e)}")
        
#         # Proper cleanup in case of error
#         if doc:
#             try:
#                 doc.close()
#             except:
#                 pass
                
#         if temp_path and os.path.exists(temp_path):
#             try:
#                 os.unlink(temp_path)
#             except:
#                 pass
                
#         return {'success': False, 'error': str(e)}

# # Your PDF URL
# pdf_url = "https://tm.com.my/iar2024/assets/files/en/tm-iar24-strategy-in-action.pdf"

# # Extract text
# print("Starting extraction...")
# result = extract_text_safe(pdf_url)

# if result['success']:
#     print(f"✅ Success! Processed {result['pages_processed']} pages (out of {result['total_pages']} total)")
#     print(f"📄 Total characters extracted: {len(result['text'])}")
#     print("\n" + "="*60)
#     print("EXTRACTED TEXT:")
#     print("="*60)
#     print(result['text'])
    
#     # Save to file
#     with open("tm_annual_report_extract.txt", "w", encoding="utf-8") as f:
#         f.write(result['text'])
#     print(f"\n💾 Text saved to 'tm_annual_report_extract.txt'")
# else:
#     print(f"❌ Extraction failed: {result['error']}")

Starting extraction...
📥 Downloading PDF...
🔍 Opening PDF document...
📄 Found 8 pages
🔄 Processing first 5 pages...
   Processing page 1...
   Processing page 2...
   Processing page 3...
   Processing page 4...
   Processing page 5...
✅ Success! Processed 5 pages (out of 8 total)
📄 Total characters extracted: 13867

EXTRACTED TEXT:

--- Page 1 ---
The growing demand of Malaysians with their digital lifestyle is reshaping Malaysia’s digital economy, further enhanced by emerging 
technologies notably AI. As Malaysia’s national connectivity and digital infrastructure provider, TM is poised to take up the role of leading 
and shaping the digital innovation to drive the digital economy and position Malaysia as a leading digital hub for ASEAN.
We are continuing with our PWR 2030 strategy which anchors on three (3) core principles: Pioneer, Win and Revitalise; aligning to our 
aspiration of becoming a Digital Powerhouse by 2030.
OUR STRATEGIC EXECUTION FOR 2024-2026: DEFEND AND BUILD
2024 ma

In [26]:
import requests
import fitz
import pytesseract
from PIL import Image
import io
import os
import uuid
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime

# Point pytesseract to your installed tesseract.exe (Windows)
pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

# ---- Global log file ----
log_file = open("processing_log.txt", "w", encoding="utf-8")

def log(message):
    """Write to log file instead of stdout to avoid recursion in Jupyter/Rich"""
    timestamp = datetime.now().strftime("%H:%M:%S")
    log_file.write(f"[{timestamp}] {message}\n")
    log_file.flush()


def extract_source_to_txt(source_url, output_dir):
    try:
        os.makedirs(output_dir, exist_ok=True)
        log(f"📥 Downloading: {source_url}")
        response = requests.get(source_url, timeout=120)
        response.raise_for_status()

        if source_url.lower().endswith(".pdf"):
            # Handle PDF
            doc = fitz.open(stream=response.content, filetype="pdf")
            total_pages = len(doc)
            log(f"📄 Total pages: {total_pages}")

            for page_num in range(total_pages):
                page = doc[page_num]
                log(f"🔎 Processing page {page_num+1}/{total_pages}...")

                # Extract digital text
                regular_text = page.get_text("text")

                # Render to image for OCR
                zoom = 2.0
                mat = fitz.Matrix(zoom, zoom)
                pix = page.get_pixmap(matrix=mat)
                img = Image.open(io.BytesIO(pix.tobytes("png")))
                ocr_text = pytesseract.image_to_string(img)

                # Combine text
                page_text = ""
                if regular_text.strip():
                    page_text += f"{regular_text.strip()}\n"
                if ocr_text.strip():
                    page_text += f"{ocr_text.strip()}\n"

                # Save with UUID
                filename = f"{uuid.uuid4()}.txt"
                page_file = os.path.join(output_dir, filename)
                with open(page_file, "w", encoding="utf-8") as f:
                    f.write(page_text)

                log(f"✅ Saved {page_file}")

            doc.close()

        else:
            # Handle Webpage
            log("🌐 Processing webpage...")
            soup = BeautifulSoup(response.text, "html.parser")

            # Remove non-content elements
            for tag in soup(["script", "style", "header", "footer", "nav", "aside", "form"]):
                tag.decompose()

            # Extract visible text
            text = soup.get_text(separator="\n", strip=True)

            # OCR text from images
            img_texts = []
            for img_tag in soup.find_all("img"):
                img_url = img_tag.get("src")
                if not img_url:
                    continue
                img_url = urljoin(source_url, img_url)
                try:
                    img_resp = requests.get(img_url, timeout=30)
                    img_resp.raise_for_status()
                    img = Image.open(io.BytesIO(img_resp.content))
                    ocr_result = pytesseract.image_to_string(img)
                    if ocr_result.strip():
                        img_texts.append(ocr_result.strip())
                except Exception as e:
                    log(f"⚠️ Could not OCR image {img_url}: {e}")

            combined_text = text
            if img_texts:
                combined_text += "\n" + "\n".join(img_texts)

            # Save as one file
            filename = f"{uuid.uuid4()}.txt"
            page_file = os.path.join(output_dir, filename)
            with open(page_file, "w", encoding="utf-8") as f:
                f.write(combined_text)

            log(f"✅ Saved webpage content to {page_file}")

    except Exception as e:
        log(f"❌ Error processing {source_url}: {e}")


In [28]:
# ---- DOMAIN ARRAYS ----
tm_product_services = [
    "https://www.tm.com.my/corporate/structure",
    "https://www.tm.com.my/news/tm-one-is-services-partner-and-managed-services-provider-of-the-year",
    "https://www.tm.com.my/",
    "https://www.tm.com.my/news/tm-rnd-ewar-collaborates-with-dhiraagu-maldives",
    "https://www.tm.com.my/iar2024/assets/files/en/tm-iar24.pdf",
    "https://www.smartmap.tm.com.my/category/products/",
    "https://www.tm.com.my/news/asean_ai_summit_2025",
    "https://tm.com.my/iar2024/assets/files/en/tm-iar24-strategy-in-action.pdf",
    "https://tm.com.my/iar2024/assets/files/en/tm-iar24-delivering-business-value-tm-global.pdf",
    "https://www.tm.com.my/news/unifi_tv_30_days_free",
    "https://www1.tm.com.my/news/unifi-business-solutions-help-msme-kickstart-online-business",
    "https://tm.com.my/news/unifi-business-empowers-hospitality-industry",
    "https://www1.tm.com.my/news/Unifi_Brings_5g_Excitement_and_Immersive_Experience",
    "https://www1.tm.com.my/news/tm-unifi-ubc-2021-webinar-reboot-5",
    "https://www1.tm.com.my/news/unifi-enhances-customers-experience-with-country-first-integrated-digital-app-myunifi-for-consumer-and-business",
    "https://tm.com.my/news/unifi-showcases-malaysias-first-tdd-pre-5G-wireless-technology",
    "https://tm.com.my/news/tm-migrates-unifi-customers-to-private-ip",
    "https://www1.tm.com.my/news/Unifi_TV_launches_original_content",
    "https://www.tm.com.my/news/first_postpaid_plan_with_netflix",
    "https://www1.tm.com.my/news/TM%25E2%2580%2599S-2520NEW-2520FIBRE-2520OPTIC-NETWORK-2520HUB",
    "https://www.tm.com.my/news/unifi-introduces-no-contract-no-rules-no-complications-the-best-of-unifi-wireless-broadband",
    "https://tm.com.my/news/tm-and-tnb-in-collaboration-to-accelerate-broadband-reach-for-the-nation",
    "https://www1.tm.com.my/news/Maxis_leverages_TM_high_speed_broadband_services",
    "https://tm.com.my/news/unifi-choices-for-different-home-needs-and-affordability",
    "https://www.tm.com.my/announcements",
    "https://www.tm.com.my/announcement/bqoes2024_css2024",
    "https://tm.com.my/news/tm_performance_q1_2025",
    "https://www.tm.com.my/news/better-internet-experience-for-streamyx-and-unifi-customers-nationwide",
    "https://www.tm.com.my/news/umobile_tm_5g_partnership",
    "https://www1.tm.com.my/news/first_postpaid_plan_with_netflix",
    "https://www.tm.com.my/news/unifi_redefines_mobile_prepaid%2520_experience_with_all_new_uni5g_wow_plans",
    "https://tm.com.my/news/tm-5g-launch-mobile-business",
    "https://www1.tm.com.my/news/tm-extends-jasa-pack-unifi-mobile-plan-to-all-pdrm-officers-and-staff-to-support-their-digital-lifestyle",
    "https://www1.tm.com.my/news/tm-expands-unifi-services-to-more-areas-in-kuching",
    "https://www1.tm.com.my/news/updates-on-unifi-mobile-5g",
    "https://www.tm.com.my/news/tm-one-teams-up-with-sap-pmc",
    "https://www.tm.com.my/news/tm-one-becomes-csp-for-government-agencies-mydigital",
    "https://tm.com.my/iar2024/assets/files/en/tm-iar24-delivering-business-value-tm-one-credence.pdf",
    "https://www1.tm.com.my/news/tm-one-5g-innovating-healthcare",
    "https://www1.tm.com.my/news/umw-tm-one-collaboration",
    "https://tm.com.my/news/tm_nct_partnership",
    "https://www1.tm.com.my/news/tm_one_launches_innovation_lab_and_enterprise_5G_lab",
    "https://www1.tm.com.my/news/tm-one-shifts-to-higher-gear-with-the-launch-of-its-10-transformative-smart-solutions",
    "https://tm.com.my/news/honor-my-appoints-tm-one-as-csp",
    "https://tm.com.my/news/tm-launch-credence",
    "https://www1.tm.com.my/news/asean_ai_summit_2025",
    "https://tm.com.my/news/tm-one-malaysia-government-data",
    "https://www.tm.com.my/news/tm_and_singtel_nxera_break_ground_for_sustainable_hyper_connected_data_centre_campus_in_johor",
    "https://tm.com.my/news/tm_singtel_nxera_develop_next_gen_data_centres",
    "http://blog.tm.com.my/3-key-components-building-malaysias-future-ready-network",
    "https://www.tm.com.my/news/tmone_pr1ma",
    "https://www.tm.com.my/news/tm-one-shifts-to-higher-gear-with-the-launch-of-its-10-transformative-smart-solutions",
    "https://www.tm.com.my/news/tm_petronas_gentari_digital_sustainable_innovation",
    "https://www.tm.com.my/news/petronas_tm_5g_lng_complex_bintulu",
    "https://www1.tm.com.my/news/TWO%2520PHASED%2520IMPLEMENTATION%2520OF%25205G",
    "https://www.tm.com.my/sites/default/files/2024-06/3.%2520Tender%2520Summary__Tender%2520IOT.pdf",
    "https://www1.tm.com.my/news/tms-innovative-digital-solutions-iot-water-energy-management-received-global-recognitions",
    "http://blog.tm.com.my/how-5g-transforming-malaysian-msmes",
    "https://www.tm.com.my/news/petronas_tm_5g_lng_complex_bintulu",
    "http://blog.tm.com.my/5g-series-banking-beyond-boundaries-leveraging-5g-advantage-enhanced-customer-centric-services",
    "https://www1.tm.com.my/news/TWO%2520PHASED%2520IMPLEMENTATION%2520OF%25205G",
    "https://www.tm.com.my/news/tm_supportive_5g_initiatives",
    "https://tm.com.my/news/tm-and-zte-malaysia-enter-into-a-strategic-partnership-to-develop-next-generation-tech-through-hybrid-cloud-5g-core-network",
    "http://blog.tm.com.my/5g-series-fueling-efficiency-5g-potential-game-changer-revolutionising-oil-gas-operations",
    "https://www1.tm.com.my/news/tm-supports-5g-development-langkawi-iocc",
    "https://www.bursamalaysia.com/bm/trade/trading_resources/listing_directory/company-profile?stock_code=4863",
    "https://www.reuters.com/markets/companies/tlmm.kl",
    "https://www.bloomberg.com/quote/486321:MK",
    "https://www.dnb.com/business-directory/company-profiles.telekom_malaysia_berhad.519bae8a597cf682f35d62895eadad9f.html",
    "https://www.crunchbase.com/organization/telekom-malaysia",
    "https://www.owler.com/company/tm",
    "https://www.zoominfo.com/c/telekom-malaysia-berhad/372151702",
    "https://finance.yahoo.com/quote/4863.KL/profile",
    "https://companiesmarketcap.com/inr/telekom-malaysia/revenue",
    "https://en.wikipedia.org/wiki/Telekom_Malaysia",
    "https://www.ft.com/content/b02fa488-907d-11e8-b639-7680cedcc421",
    "https://www.wsj.com/market-data/quotes/MY/XKLS/4863/company-people",
    "https://seekingalpha.com/symbol/MYTEF/income-statement",
    "www.morningstar.com/stocks/xkls/4863/quote",
    "https://www.gurufocus.com/term/ROE/MYTEF",
    "https://pitchbook.com/profiles/company/63229-42",
    "https://www.marketscreener.com/quote/stock/TELEKOM-MALAYSIA-6491267/company-shareholders",
    "https://www.fitchratings.com/research/corporate-finance/fitch-affirms-telekom-malaysia-at-bbb-withdraw-ratings-13-05-2025",
    "https://www.ram.com.my/pressrelease/?prviewid=6981",
    "https://www.emis.com/php/company-profile/MY/Telekom_Malaysia_Berhad_en_1663424.html",
    "https://www.lowyat.net/2025/366122/tm-revamps-unifi-tv-service-now-available-to-non-unifi-customers/",
    "https://www.datacenterjournal.com/telekom-malaysia-berhad-tm/",
    "https://www.malaysianwireless.com/wp-content/uploads/2019/03/MCMC-2018-2020-MSAP-Access-Pricing.pdf",
    "https://ejournal.aibpmjournals.com/index.php/APJME/article/viewFile/1429/1301",
    "https://www.gsma.com/solutions-and-impact/technologies/internet-of-things/wp-content/uploads/2019/09/TM-ONE-case-study-c-1.pdf",
    "https://www.researchandmarkets.com/reports/5893661/malaysia-data-center-market-competition",
    "https://www.mordorintelligence.com/industry-reports/malaysia-data-center-market",
    "https://www.lowyat.net/2025/355249/double-data-speeds-from-rm169-with-unifis-newest-broadband-bundle/",
    "https://www.comparehero.my/technology/articles/unifi-the-honest-review",
    "https://www.imoney.my/articles/tm-unifi-broadband-packages-for-your-home",
    "https://www.malaysianwireless.com/2024/09/telekom-malaysia-unifi-business-fibre-package-from-rm129-month/",
    "https://www.soyacincau.com/2018/07/12/telekom-malaysia-speed-upgrade-800mbps-unifi-basic-rm79/",
    "https://www.malaysianwireless.com/2019/09/unifi-air-broadband-rm79-month-unlimited-data/",
    "https://www.lowyat.net/2020/226162/unifi-turbo-speed-upgrade-malaysia-telekom-complaints/",
    "https://www.lowyat.net/2021/237227/unifi-launches-unifi-mobile-package-for-spm-stpm-students-rm20-for-15gb/",
    "https://www.soyacincau.com/2019/02/18/unifi-mobile-rm99-unlimited-plan-permanent/",
    "https://www.thestar.com.my/tech/tech-news/2023/12/28/tm-one-collaborates-with-nvidia-to-launch-ai-cloud-services-in-malaysia-by-early-2024",
    "https://www.digitalnewsasia.com/business/tm-one-launch-3rd-data-center-q2-2017-cyberjaya",
    "https://www.comparehero.my/technology/articles/home-broadband-packages-comparison-malaysia",
    "https://www.malaysianwireless.com/2024/04/unifi-home-broadband-plans-2024-comparison/"
]

tm_corporate_info = [
    "https://www.tm.com.my/corporate/structure",
    "https://www.tm.com.my/investor-relations/financial-information/annual-reports",
    "https://www.tm.com.my/investor-relations/resources",
    "https://www.tm.com.my/corporate/corporate-governance",
    "https://www.tm.com.my/investor-relations/financial-information",
    "https://www.tm.com.my/iar2024/assets/files/en/tm-iar24.pdf",
    "https://www.tm.com.my/investor-relations/resources/analyst-presentation",
    "https://www.tm.com.my/investor-relations",
    "https://www.tm.com.my/corporate/our-people/ahmad-taufek-omar",
    "https://www.tm.com.my/taxonomy/term/114",
    "https://www.tm.com.my/corporate/management-team",
    "https://tm.com.my/sites/default/files/2022-11/TM-CGOS_2021.pdf",
    "https://ytm.tm.com.my/board-trustees",
    "https://www.tm.com.my/corporate/our-people",
    "https://www.tm.com.my/sites/default/files/Documents/TM%2520BOD%2520Charter%2520V.9.2%2520%2528abridge%2529.pdf",
    "https://www.tm.com.my/sites/default/files/Documents/TM-CG-Board_Subs_Remuneration_Framework_20190103.pdf",
    "https://www.tm.com.my/investor-relations/financial-information/financial-performance",
    "https://www.tm.com.my/investor-relations/financial-information/quarterly-results",
    "https://www.tm.com.my/node",
    "https://www1.tm.com.my/news/q2_financial_results",
    "https://tm.com.my/news/tm_performance_q1_2025",
    "https://www.tm.com.my/iar2024/",
    "https://www.tm.com.my/investor-relations/news-events/events",
    "https://tm.com.my/iar2024/assets/files/en/tm-fr24.pdf",
    "https://www.tm.com.my/sites/default/files/2022-12/TM-IAR2021.pdf",
    "https://www.tm.com.my/sites/default/files/2023-05/%255BOur%2520Material%2520Matters%255D%2520TM%2520IAR2022.pdf",
    "https://www1.tm.com.my/sites/default/files/2025-06/TM%2520IAR2024_Our%2520Reporting%2520Journey.pdf",
    "https://tm.com.my/sites/default/files/2024-05/TM_IAR23_E_Our%2520Reporting%2520Journey.pdf",
    "https://www.tm.com.my/sites/default/files/2025-06/TM%2520IAR2024_Sustainability%2520At%2520TM.pdf",
    "https://www.tm.com.my/iar2024/assets/files/en/tm-iar24-words-from-our-leaders-dato-zainal-abidin-putih.pdf",
    "https://www.tm.com.my/iar2024/assets/files/en/tm-iar24-words-from-our-leaders-amar-huzaimi-md-deris.pdf",
    "https://www.bursamalaysia.com/bm/trade/trading_resources/listing_directory/company-profile?stock_code=4863",
    "https://www.spglobal.com/ratings/en/regulatory/article/-/view/type/HTML/id/3411656",
    "https://simplywall.st/stocks/my/telecom/klse-tm/telekom-malaysia-berhad-shares/ownership",
    "https://www.sec.gov/Archives/edgar/data/1468091/000119312517107561/d318647d20f.htm",
    "https://stockanalysis.com/quote/klse/TM/",
    "https://www.aberdeeninvestments.com/en-ch/institutional/insights-and-research/emerging-markets-income-equity-why-telekom-malaysia-stands-out-from-the-crowd",
    "https://www.ram.com.my/pressrelease/?prviewid=6333",
    "https://www.futunn.com/en/stock/MYTEF-US/company",
    "https://cbonds.com/news/3308651/",
    "https://www.researchgate.net/publication/239765735_Structural_capital_and_its_effect_on_organizational_performance_A_case_study_of_Telekom_Malaysia_Berhad_TM_headquarters",
    "https://www.proactiveinvestors.co.uk/companies/news/1014169/telekom-malaysia-unveils-new-dividend-policy-amidst-stronger-earnings-1014169.html",
    "https://www.theedgemarkets.com/article/telekom-malaysia-reports-rm38618-mil-net-profit-q2-2023",
    "https://mtsfb.org.my/wp-content/uploads/2024/09/MTSFB-AR-2021_pages_lowres.pdf",
    "https://www.reuters.com/markets/companies/TLMM.KL/profile/",
    "https://www.ft.com/content/telekom-malaysia-berhad-4863",
    "https://www.bnm.gov.my/documents/20124/938025/FSF+Review+2014-Box+2.3+Growth+and+Performance+of+the+Communications+and+Multimedia+Sector.pdf/f0196232-a549-065d-d9b9-d29227181c4e?t=1599818815152",
    "https://www.researchgate.net/publication/283109345_The_Role_of_Brand_Loyalty_The_Case_Study_of_Telekom_Malaysia",
    "https://www.google.com/finance/quote/TM:KLSE",
    "https://www.datacenters.com/companies/telekom-malaysia",
    "https://www.nst.com.my/business/corporate/2019/06/500171/tm-launches-new-strategy-focus-core-businesses"
]

tm_customer_support = [
    "https://www.tm.com.my/corporate/corporate-contact",
    "https://www.tm.com.my/news/relief_measures_putra_heights",
    "https://www1.tm.com.my/news/unifi-enhances-customers-experience-with-country-first-integrated-digital-app-myunifi-for-consumer-and-business",
    "https://www1.tm.com.my/news/tm_scicom_ai_powered_customer_experience",
    "https://tm.com.my/news/tm-continues-services-during-fmco",
    "https://www1.tm.com.my/news/tm-transformation-reinforces-customer-centricity",
    "https://www.tm.com.my/iar2024/assets/files/en/tm-iar24-delivering-business-value-unifi.pdf",
    "https://www1.tm.com.my/news/TM-response-to-the-national-crisis-and-reassurance",
    "https://tm.com.my/news/tm-5g-launch-mobile-business",
    "https://www1.tm.com.my/news/TM%25E2%2580%2599S%2520NEW%2520FIBRE%2520OPTIC%2520NETWORK%2520HUB",
    "https://www.tm.com.my/downloads/e-invoicing-faq-english",
    "https://e-invoice.tm.com.my/faq.html",
    "https://www.smartmap.tm.com.my/faq/",
    "https://www.tm.com.my/sites/default/files/Documents/FAQ_ST_8%2525_2024.pdf",
    "https://www.tm.com.my/node/1996",
    "https://www.tm.com.my/node/925",
    "https://www.tm.com.my/node/574",
    "https://www.tm.com.my/node/937",
    "https://www.tm.com.my/node/919",
    "https://www.smartmap.tm.com.my/contact-us/",
    "https://www.tm.com.my/corporate/corporate-contact",
    "https://www.tm.com.my/investor-relations/contact",
    "https://www.tm.com.my/business-with-tm/overview",
    "https://tmbizonline.tm.com.my/",
    "https://www.tm.com.my/node/865",
    "https://tmbizonline.tm.com.my/b2bapps/media/faTMBizOnlin_flyer_rev16072014.pdf",
    "https://www.tm.com.my/node/841",
    "https://tm.com.my/news/tm-one-5g-innovating-healthcare",
    "https://www.smartmap.tm.com.my/wp-content/uploads/2021/12/SmartMap-API-SST-Services-Schedule-to-Integrated-MSA.pdf",
    "https://www1.tm.com.my/news/unisza-embarks-on-a-digital-transformation-journey-with-tm-one-dedicated-premium-network-connectivity",
    "https://www.tm.com.my/sites/default/files/2022-12/TM-IAR2021.pdf",
    "https://www.smartmap.tm.com.my/wp-content/uploads/2022/10/SST-for-SmartMap-Data-Services-Ver.-1-2022-July-31-2022.pdf",
    "https://www.tm.com.my/news/first_ai_music_platform",
    "https://www.tm.com.my/sites/default/files/2024-06/StartLink_Summary.pdf",
    "https://www.tm.com.my/news/GPUaaS_empower_ai",
    "https://tm.com.my/iar2023/assets/files/TM_IAR23.pdf"
]

tm_sustainability_esg = [
    "https://tm.com.my/iar2023/assets/files/TM_IAR23_Our_Sustainability_Journey.pdf",
    "https://www.tm.com.my/sites/default/files/2025-06/TM%2520IAR2024_Sustainability%2520At%2520TM.pdf",
    "https://www.tm.com.my/sustainability/policies-guidelines",
    "https://tm.com.my/iar2024/assets/files/en/tm-iar24-driving-sustainable-and-meaningful-impact.pdf",
    "https://www.tm.com.my/sustainability",
    "https://www.tm.com.my/sites/default/files/Documents/TM-Sustainability-Policy.pdf",
    "https://www.tm.com.my/sustainability/our-approach",
    "https://www.tm.com.my/sites/default/files/2025-06/TM%2520IAR2024_Bursa%2520Malaysia%2520ESG%2520Reporting.pdf",
    "http://blog.tm.com.my/tech-driven-sustainability-renewed-hope-sustainably-developed-world",
    "https://www.tm.com.my/iar2022/files/Our_Sustainability_Journey.pdf",
    "https://tm.com.my/sustainability/our-pillars",
    "https://tm.com.my/news/TM_further_accelerates_its_sustainable_development",
    "https://www.tm.com.my/sites/default/files/2025-06/TM%2520IAR2024_Bursa%2520Malaysia%2520ESG%2520Reporting.pdf",
    "https://www1.tm.com.my/sites/default/files/2025-06/TM%2520IAR2024_Our%2520Material%2520Matters.pdf",
    "https://www.tm.com.my/iar2024/assets/files/en/tm-iar24-words-from-our-leaders-dato-zainal-abidin-putih.pdf",
    "https://tm.com.my/iar2023/assets/files/TM_IAR23_Our_Sustainability_Journey.pdf",
    "https://www.tm.com.my/iar2024/assets/files/en/tm-iar24-words-from-our-leaders-ahmad-fairus-rahim.pdf",
    "https://www.tm.com.my/sustainability/our-impact",
    "https://blog.tm.com.my/media/149/download",
    "http://blog.tm.com.my/transforming-csr-sustainable-strategies-empowering-underserved-communities",
    "https://www.tm.com.my/sustainability/our-impact",
    "https://www.tm.com.my/sites/default/files/Documents/%255BCommunity-Development%255D-TM-IAR2022.pdf",
    "https://www.tm.com.my/news/tm_petronas_gentari_digital_sustainable_innovation",
    "https://www.tm.com.my/corporate/achievements",
    "https://www.tm.com.my/sites/default/files/Documents/YTM_Social_Impact_Grant_2024-2.pdf",
    "https://www1.tm.com.my/news/TM_Enhanced_Mobile_Connectivity_Nationwide",
    "https://tm.com.my/sustainability/our-pillars",
    "https://www.tm.com.my/sites/default/files/2023-05/%255BESG%2520Roadmaps%2520and%2520Commitments%255D%2520TM%2520IAR2022.pdf",
    "https://tm.com.my/news/TM_further_accelerates_its_sustainable_development",
    "https://www.tm.com.my/sustainability/our-impact",
    "https://tm.com.my/iar2023/assets/files/TM_IAR23_Our_Sustainability_Journey.pdf",
    "https://www.tm.com.my/sites/default/files/Documents/%255BEnvironmental-Quality%255D-TM-IAR2022.pdf",
    "https://tm.com.my/news/tm-data-centres-now-powered-by-clean-energy",
    "https://ytm.tm.com.my/malaysian-students-turn-1000-plastic-bottles-art-greener-future"
]

tm_digital_transformation = [
    "http://blog.tm.com.my/four-imperatives-successful-digital-transformation",
    "http://blog.tm.com.my/tm-leads-charge-malaysias-digital-transformation-journey",
    "https://tm.com.my/iar2024/assets/files/en/tm-iar24-delivering-business-value-tm-one-credence.pdf",
    "http://blog.tm.com.my/orchestrating-concerto-digital-transformation-through-wholesale-business",
    "http://blog.tm.com.my/empowering-digital-transformation-malaysian-msmes",
    "https://www.tm.com.my/news/asean_ai_summit_2025",
    "https://www.tm.com.my/news/tm_petronas_gentari_digital_sustainable_innovation",
    "https://www.tm.com.my/news/unisza-embarks-on-a-digital-transformation-journey-with-tm-one-dedicated-premium-network-connectivity",
    "https://tm.com.my/iar2024/assets/files/en/tm-iar24-strategy-in-action.pdf",
    "http://blog.tm.com.my/leaping-beyond-pandemic-top-business-priorities-smart-rebound",
    "https://www1.tm.com.my/news/tm_one_launches_innovation_lab_and_enterprise_5G_lab",
    "https://tm.com.my/iar2024/assets/files/en/tm-iar24-strategy-in-action.pdf",
    "http://blog.tm.com.my/everyday-innovations-reimagining-permits-work",
    "https://tm.com.my/news/TM_ZTE_Partnership",
    "https://ytm.tm.com.my/empowering-innovation-and-education-tech-titans-challenge",
    "http://blog.tm.com.my/innovation-imperative-unlocking-sustainable-economic-growth",
    "https://www.tm.com.my/news/tm-rnd-wins-aces-awards-2022",
    "https://www1.tm.com.my/news/TM_fiberhome_launch_innovation_lab",
    "https://tm.com.my/news/tm-one-becomes-csp-for-government-agencies-mydigital",
    "https://www1.tm.com.my/news/geran_digital_PMKS_madani",
    "https://www1.tm.com.my/news/tm-one-5g-innovating-healthcare",
    "http://blog.tm.com.my/digital-playbook-essential-strategies-malaysian-msmes-thrive",
    "https://www.tm.com.my/iar2024/assets/files/en/tm-iar24-delivering-business-value-unifi.pdf",
    "https://www.tm.com.my/iar2024/assets/files/en/tm-iar24-words-from-our-leaders-ahmad-fairus-rahim.pdf",
    "https://ytm.tm.com.my/facilitating-growth-underserved-communities"
]

telecom_industry_knowledge = [
    "https://0308899econs.wordpress.com/2013/06/08/17/",
    "https://pdfcoffee.com/strategic-analysis-of-malaysian-telecommunication-industry-pdf-free.html",
    "https://www.researchgate.net/publication/347219082_The_Effect_of_Service_Quality_Satisfaction_and_Loyalty_toward_Customer_Retention_in_the_Telecommunication_Industry",
    "https://kwpublications.com/papers_submitted/16402/the-effect-of-service-quality-satisfaction-and-loyalty-toward-customer-retention-in-the-telecommunication-industry.pdf",
    "https://www.researchgate.net/publication/343888529_The_Influence_of_Critical_Success_Factors_of_Lean_Six_Sigma_towards_Supply_Chain_Performance_in_Telecommunication_Industry_Malaysia",
    "https://ojs.excelingtech.co.uk/index.php/IJSCM/article/view/3197",
    "https://dr.ntu.edu.sg/server/api/core/bitstreams/531d7d6b-1-16da-4b06-8b80-d95c8e383e39/content",
    "http://eprints.utar.edu.my/2652/1/Completed_FYP.pdf",
    "https://www.tandfonline.com/doi/full/10.1080/23311916.2022.2124940",
    "https://www.glassdoor.ie/Job/jobs.htm?sc.occupationParam=Engineering%2BTeam%2BLead%2B-2528Telecommunication%2BIndustry%2529&sc.locationSeoString=Malaysia&locId=170&locT=N",
    "https://www.mordorintelligence.com/industry-reports/malaysia-telecom-market",
    "https://www.ey.com/en_my/insights/telecommunications/top-10-risks-for-telecommunications-in-2025",
    "https://www.analysysmason.com/research/content/regional-forecasts-/malaysia-telecoms-forecasts-rdrp0/",
    "https://finance.yahoo.com/news/malaysia-telecoms-industry-report-2022-124300676.html",
    "https://www.mckinsey.com/industries/technology-media-and-telecommunications/our-insights/the-future-of-telcos-mapping-the-routes-to-renewed-success",
    "https://www.researchandmarkets.com/reports/5764230/malaysia-telecom-market-share-analysis?srsltid=AfmBOoqGE3sV_I2PA2F4LhpV0LqBUaOML7Ud0rvP6Ujq3NZCJQ7mL7xY",
    "https://telecomlead.com/5g/what-are-5g-network-investment-trends-in-malaysia-120112",
    "https://www.lucintel.com/Malaysia/telecom-cloud-market.aspx",
    "https://www.linkedin.com/pulse/malaysia-enterprise-b2b-telecom-market-type-38woe",
    "https://mcmc.gov.my/skmmgovmy/media/General/pdf/3Q16-infog.pdf",
    "https://sems.mcmc.gov.my/sems/contents/show/inline/1/content/en?lang=ms",
    "https://www.crunchbase.com/organization/telekom-malaysia/news_and_analysis",
    "https://www.freemalaysiatoday.com/category/tag/telekom-malaysia",
    "https://www.cbinsights.com/investor/telekom-malaysia",
    "https://en.wikipedia.org/wiki/Telekom_Malaysia",
    "https://www.livemint.com/topic/amp-telekom-malaysia",
    "https://www.zoominfo.com/c/telekom-malaysia-berhad/372151702",
    "https://www.freemalaysiatoday.com/category/tag/koperasi-telekom-malaysia",
    "https://www.exportgenius.in/company/akamai-bv-co-telekom-malaysia/0beff8d8553b00a5e64cfb245c19f84b",
    "https://www.scribd.com/doc/45791326/Telekom-Malaysia",
    "https://asia.nikkei.com/business/mytv-set-to-go-digital-by-mid-2015",
    "https://www.thestar.com.my/business/business-news/2025/09/29/tm-invests-in-growth",
    "https://www.thestar.com.my/business/business-news/2025/06/14/the-skys-the-limit-for-tm",
    "https://www.thestar.com.my/business/business-news/2025/09/03/network-infrastructure-work-to-bolster-tm",
    "https://www.thestar.com.my/business/business-news/2025/09/01/tm-quarterly-net-profit-increases-to-rm403mil",
    "https://www.thestar.com.my/business/business-news/2025/09/22/tm-joins-consortium-to-develop-cable-system-connecting-asean-nations-to-japan",
    "https://www.thestar.com.my/business/business-news/2025/08/29/tms-2q-net-profit-up-17-to-rm403mil-declares-125-sen-dividend",
    "https://www.thestar.com.my/Business/Marketwatch/Stocks?qcounter=TM",
    "https://www.thestar.com.my/esg/2025/08/29/beyond-the-cloud",
    "https://www.thestar.com.my/business/business-news/2025/05/29/tm-delivers-steady-first-quarter-performance",
    "https://www.thestar.com.my/business/business-news/2025/02/26/tm-to-spend-big-on-broadband-expansion",
    "https://www.bernama.com/en/region/news.php?id=2468303",
    "https://www.bernama.com/en/business/news.php?id=2470186",
    "https://www.bernama.com/en/business/news.php?id=2427805",
    "https://bernama.com/en/business/news.php?id=2368134",
    "https://www.bernama.com/en/news.php?id=2304209",
    "https://www.bernama.com/en/business/news.php?id=2428253",
    "https://bernama.com/en/business/news.php?id=2396437",
    "https://www.bernama.com/en/business/news.php?id=2147633",
    "https://www.bernama.com/en/business/news.php?id=2367943",
    "https://www.bernama.com/en/news.php?id=2333663",
    "https://www.tm.com.my/iar2024/assets/files/en/tm-iar24.pdf",
    "https://tm.com.my/iar2024/assets/files/en/tm-iar24-2024-snapshot.pdf",
    "https://tm.com.my/iar2024/assets/files/en/tm-iar24-delivering-business-value-tm-global.pdf",
    "https://www.kenanga.com.my/wp-content/uploads/2025/02/TM-250226-4QFY24-RN-Kenanga.pdf",
    "https://cdn1.i3investor.com/my/files/st88k/4863_TM/pt/KENANGA/4863_TM_KENANGA_2025-02-05_BUY_7.53_TelekomMalaysiaQuadPlay5GFiberDataCenterampGPU_2071560333.pdf",
    "https://www.tmforum.org/wp-content/uploads/2021/02/TM-Forum-Excellence-Awards-Case-Study-AID-Telekom-Malaysia.pdf",
    "https://www.tm.com.my/sites/default/files/2022-12/TM-IAR2021.pdf",
    "https://tm.listedcompany.com/misc/NR_1Q_2024_Financial_Results.pdf",
    "https://www.kenanga.com.my/wp-content/uploads/2024/05/TM-240509-CU-Kenanga.pdf",
    "https://tm.com.my/iar2024/assets/files/en/tm-iar24-value-creation-business-model.pdf",
    "https://archive.data.gov.my/data/dataset/telecommunications-statistics-malaysia",
    "https://archive.data.gov.my/data/ms_MY/dataset/telecommunications-statistics-malaysia/resource/f63bc771-a1bd-46d2-9c1e-7ea0a390d04a",
    "https://www.ceicdata.com/en/malaysia/telecommunication-statistics-summary",
    "https://datahub.itu.int/data/?e=MYS&Markets=Spectrum&Connectivity=Use&Sustainability=Economy",
    "https://data.worldbank.org/indicator/IT.NET.USER.ZS?locations=MY",
    "https://www.analysysmason.com/research/content/country-reports/malaysia-country-report-rdrp0/",
    "https://datarade.ai/data-categories/telecom-data/datasets",
    "https://www.reportlinker.com/dataset/e0f9b525ae6c8ca248757d37f51874e02df84095",
    "https://www.mordorintelligence.com/industry-reports/malaysia-telecom-market",
    "https://data360.worldbank.org/en/dataset/OECD_BROADBAND"
]

# Put all domains in one dict {folder: urls[]}
all_domains = {
    "TM_Product_Services": tm_product_services,
    "TM_Corporate_Info": tm_corporate_info,
    "TM_Customer_Support": tm_customer_support,
    "TM_Sustainability_ESG": tm_sustainability_esg,
    "TM_Digital_Transformation": tm_digital_transformation,
    "Telecom_Industry_Knowledge": telecom_industry_knowledge,
}

# ---- LOOP THROUGH ALL DOMAINS ----
for domain_name, urls in all_domains.items():
    for url in urls:
        _ = extract_source_to_txt(url, os.path.join("output", domain_name))

In [3]:
import os
os.environ["DEEPSEEK_API_KEY"] = ""

In [ ]:
import os
import json
import glob
from pathlib import Path
from datetime import datetime
import time
import re

# Disable all output formatting
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_VERBOSE"] = "false"

# Set up DeepSeek API
os.environ["DEEPSEEK_API_KEY"] = ""

# Import after setting env variables
from langchain_openai import ChatOpenAI

# Create log file to avoid stdout issues
log_file = open("processing_log.txt", "w", encoding="utf-8")

def log(message):
    """Write to log file instead of stdout to avoid rich library recursion"""
    timestamp = datetime.now().strftime("%H:%M:%S")
    log_file.write(f"[{timestamp}] {message}\n")
    log_file.flush()

def split_into_sections(text, min_words=150):
    """Split text into meaningful sections with overlap"""
    sections = re.split(r'\n\s*\n|\n', text)
    
    cleaned_sections = []
    for section in sections:
        section = section.strip()
        word_count = len(section.split())
        if word_count >= min_words:
            cleaned_sections.append(section)
    
    if not cleaned_sections:
        words = text.split()
        chunk_size = 300
        overlap = 50
        
        for i in range(0, len(words), chunk_size - overlap):
            chunk = ' '.join(words[i:i+chunk_size])
            if len(chunk.split()) >= min_words:
                cleaned_sections.append(chunk)
    
    if not cleaned_sections:
        cleaned_sections = [text]
    
    return cleaned_sections

def clean_text(text):
    """Remove special symbols and clean text"""
    text = text.replace('*', '').replace('#', '').replace('`', '')
    text = text.replace('_', '').replace('~', '').replace('|', '')
    text = text.replace('[', '').replace(']', '').replace('{', '').replace('}', '')
    text = ' '.join(text.split())
    return text.strip()

def extract_multiple_contexts(text, domain, llm, max_retries=2):
    """Extract MULTIPLE contexts from text"""
    
#     for attempt in range(max_retries):
#         try:
#             prompt = f"""You are analyzing text from the {domain} domain. Extract EVERY SINGLE piece of information as separate contexts.

# Text to analyze:
# {text}

# CRITICAL INSTRUCTIONS:
# 1. Read through the ENTIRE text carefully
# 2. Identify EVERY distinct concept, fact, explanation, example, or piece of information
# 3. Create a SEPARATE context for EACH distinct piece
# 4. ENSURE COMPLETE COVERAGE - every sentence and idea must be captured
# 5. Each context must be:
#    - ACCURATE: Factually correct and precise
#    - COMPREHENSIVE: Cover that specific topic thoroughly
#    - DETAILED: Appropriate depth in ONE paragraph
#    - PLAIN TEXT ONLY: No symbols, markdown, or formatting

# Format EXACTLY like this:
# CONTEXT_1: [First context paragraph]
# CONTEXT_2: [Second context paragraph]
# CONTEXT_3: [Third context paragraph]

# Extract AT LEAST one context for every 50-100 words. For 500 words, extract at least 5-10 contexts."""

    for attempt in range(max_retries):
        try:
            prompt = f"""You are analyzing text from the {domain} domain. Extract EVERY distinct piece of information as separate contexts.

Before producing any output, read the ENTIRE text in {text} from start to finish. Do not start writing contexts until you have completed reading the whole input.

CRITICAL INSTRUCTIONS:

1. Read the entire text first. Do not produce any output until you have finished reading.
2. Identify every distinct concept, fact, claim, explanation, example, or piece of information in the text.
3. Decide the optimal number of contexts yourself based on the text's semantic structure and logical boundaries. Do NOT split mechanically by a fixed word-count (for example “every 50-100 words”) — that causes context cutting. Instead, use adaptive granularity so each context represents a single coherent idea or a closely related group of ideas.
4. Each context must be a single paragraph in plain text only (no markdown, symbols, bullet lists, or extra formatting) and must be comprehensive while not exceeding 1000 words.
5. Ensure COMPLETE COVERAGE: every sentence and idea in the input must be captured. Combine closely related sentences or lines into the same context when they form one idea. If the same idea appears multiple times, consolidate and capture it once in a single, comprehensive context that references all occurrences.
6. Each context paragraph must be accurate, precise, and sufficiently detailed (one paragraph per context). Avoid splitting one idea across multiple contexts.
7. Format the output EXACTLY like this (plain text only, numbered sequentially with no gaps):
   CONTEXT_1: [First context paragraph]
   CONTEXT_2: [Second context paragraph]
   CONTEXT_3: [Third context paragraph]
   ...continue until all distinct ideas are covered.
8. Do not include any prefatory or trailing text, analysis, comments, metadata, or explanations outside the CONTEXT_n lines. Output only the CONTEXT lines.
9. If the input contains ambiguous statements, make a best-effort interpretation and include that interpretation clearly within the relevant context paragraph; do not ask clarifying questions.
10. If a single coherent idea requires more than 1000 words, split that idea into the fewest possible contexts while keeping each paragraph coherent and ≤1000 words.
11. If there is no extractable information, output exactly:
    CONTEXT_1: No extractable content.

Processing note: After you finish reading, begin numbering at CONTEXT_1 and continue sequentially until every distinct idea from {text} is represented.
"""

            response = llm.invoke(prompt)
            content = response.content.strip()
            
            contexts = []
            lines = content.split('\n')
            current_context = ""
            
            for line in lines:
                line = line.strip()
                if line.startswith('CONTEXT_'):
                    if current_context:
                        cleaned = clean_text(current_context)
                        if len(cleaned.split()) >= 20:
                            contexts.append(cleaned)
                    parts = line.split(':', 1)
                    if len(parts) > 1:
                        current_context = parts[1].strip()
                    else:
                        current_context = ""
                elif current_context:
                    current_context += " " + line
            
            if current_context:
                cleaned = clean_text(current_context)
                if len(cleaned.split()) >= 20:
                    contexts.append(cleaned)
            
            if contexts:
                return contexts
            
            if attempt < max_retries - 1:
                log(f"    No contexts found, retrying ({attempt + 1}/{max_retries})...")
                time.sleep(1)
                
        except Exception as e:
            log(f"    Error extracting contexts (attempt {attempt + 1}): {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2)
    
    return []

def generate_question(context, domain, llm, max_retries=2):
    """Generate question for a context"""
    
    for attempt in range(max_retries):
        try:
            prompt = f"""Based on this context about {domain}, create ONE clear, specific question.

Context:
{context}

Requirements:
- Practical and realistic question
- Clear and specific
- Tests real understanding
- Matches context complexity

Return ONLY the question text."""

            response = llm.invoke(prompt)
            question = clean_text(response.content.strip())
            
            if question and len(question.split()) >= 3:
                return question
            
            if attempt < max_retries - 1:
                time.sleep(1)
                
        except Exception as e:
            log(f"    Error generating question (attempt {attempt + 1}): {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2)
    
    return None

def process_section(section, domain, llm, section_num, total_sections, max_contexts=20):
    """Process a section and extract Q&A pairs"""
    log(f"  Section {section_num}/{total_sections} ({len(section.split())} words)")
    
    log("    Extracting contexts...")
    contexts = extract_multiple_contexts(section, domain, llm)
    
    if not contexts:
        log("    No contexts extracted")
        return []
    
    if len(contexts) > max_contexts:
        log(f"    Found {len(contexts)} contexts (limiting to {max_contexts})")
        contexts = contexts[:max_contexts]
    else:
        log(f"    Found {len(contexts)} contexts")
    
    results = []
    failed_count = 0
    
    for i in range(len(contexts)):
        context = contexts[i]
        context_num = i + 1
        
        try:
            log(f"      Context {context_num}/{len(contexts)}: Generating question...")
            
            question = generate_question(context, domain, llm)
            
            if question:
                json_obj = {
                    "instruction": question,
                    "context": f"You are an expert in the domain of {domain}.",
                    "response": context
                }
                results.append(json_obj)
                log(f"      Q&A pair created")
            else:
                failed_count += 1
                log(f"      Failed to generate question")
            
            time.sleep(0.5)
            
        except Exception as e:
            failed_count += 1
            log(f"      Error in context {context_num}: {str(e)}")
            continue
    
    log(f"    Section complete: {len(results)} success, {failed_count} failed")
    return results

def process_text_file(file_path, domain, llm, max_sections=50):
    """Process a single text file"""
    log(f"Reading: {Path(file_path).name}")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except Exception as e:
        log(f"Read error: {str(e)}")
        return None
    
    word_count = len(content.split())
    log(f"  Loaded {word_count} words")
    
    expected_min_contexts = max(word_count // 100, 5)
    log(f"  Target: Extract at least {expected_min_contexts} contexts")
    
    log("  Splitting into sections...")
    sections = split_into_sections(content, min_words=150)
    
    if len(sections) > max_sections:
        log(f"  Created {len(sections)} sections (limiting to {max_sections})")
        sections = sections[:max_sections]
    else:
        log(f"  Created {len(sections)} sections")
    
    all_results = []
    
    for section_idx in range(len(sections)):
        section = sections[section_idx]
        section_num = section_idx + 1
        
        try:
            results = process_section(section, domain, llm, section_num, len(sections))
            
            if results:
                all_results.extend(results)
            
            time.sleep(1)
            
        except Exception as e:
            log(f"  Error processing section {section_num}: {str(e)}")
            continue
    
    if expected_min_contexts > 0:
        coverage_ratio = len(all_results) / expected_min_contexts
        
        if coverage_ratio < 0.7:
            log(f"  WARNING: Only extracted {len(all_results)} contexts (expected ~{expected_min_contexts})")
        else:
            log(f"  Good coverage: {len(all_results)} contexts extracted")
    
    log(f"TOTAL: {len(all_results)} Q&A pairs created")
    return all_results

def main():
    """Main processing function"""
    log("="*60)
    log("TEXT TO JSON CONVERTER - DeepSeek AI")
    log("="*60)
    
    log("Initializing DeepSeek LLM...")
    try:
        llm = ChatOpenAI(
            model="deepseek-chat",
            openai_api_key=os.environ["DEEPSEEK_API_KEY"],
            openai_api_base="https://api.deepseek.com",
            temperature=0.7,
            max_tokens=3000,
            request_timeout=60,
            verbose=False
        )
        log("LLM ready")
    except Exception as e:
        log(f"LLM init failed: {str(e)}")
        return
    
    log("\nSearching for text files...")
    text_files = glob.glob("output/*/*.txt")
    
    if not text_files:
        log("No files found in output/*/*.txt")
        return
    
    log(f"Found {len(text_files)} files\n")
    
    total_processed = 0
    total_failed = 0
    total_qa_pairs = 0
    
    for file_idx in range(len(text_files)):
        file_path = text_files[file_idx]
        file_num = file_idx + 1
        
        log("-" * 60)
        log(f"FILE {file_num}/{len(text_files)}")
        log("-" * 60)
        
        try:
            path_parts = Path(file_path).parts
            domain = path_parts[-2]
            uuid = Path(file_path).stem
            
            log(f"Domain: {domain}, UUID: {uuid}")
            
            json_objects = process_text_file(file_path, domain, llm)
            
            if not json_objects or len(json_objects) == 0:
                log("No valid results")
                total_failed += 1
                log("")
                continue
            
            output_dir = Path(f"chunked/{domain}")
            output_dir.mkdir(parents=True, exist_ok=True)
            output_file = output_dir / f"{uuid}.json"
            
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(json_objects, f, indent=2, ensure_ascii=False)
            
            log(f"SAVED: {output_file}")
            log(f"Contains: {len(json_objects)} Q&A pairs\n")
            
            total_processed += 1
            total_qa_pairs += len(json_objects)
            
        except Exception as e:
            log(f"Error: {str(e)}")
            total_failed += 1
            log("")
            continue
    
    log("=" * 60)
    log("SUMMARY")
    log("=" * 60)
    log(f"Files processed: {total_processed}")
    log(f"Files failed: {total_failed}")
    log(f"Total Q&A pairs: {total_qa_pairs}")
    log("=" * 60)
    
    log_file.close()

if __name__ == "__main__":
    main()
    print("Processing complete! Check 'processing_log.txt' for details.")